# Make Detections with Model

In [1]:
import pickle 
import csv
import os
import numpy as np
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [4]:
#initiating the model
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                                 
        
        # 2. Right hand
        #mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
         #                        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
          #                       mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
           #                      )

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [5]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [6]:
with open(r'C:\Users\hagar\OneDrive - mail.tau.ac.il\Desktop\Stage\LPC_2022\Hand decoder\Position\code\position.pkl', 'rb') as f:
    model = pickle.load(f)

In [7]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [8]:
landmarks = ['predicted_class','probablitiy_of_pred']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [12]:
fn = 'word_h0_01' #name of the video we want to know its positions

In [13]:
file_path = f'C:/Users/hagar/OneDrive - mail.tau.ac.il/Desktop/Stage/LPC_2022/Hand decoder/Position/results/predicted_csv/{fn}_pose_estimation_by_frames.csv'

with open(file_path,mode='w', newline='') as f: 
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [14]:
cap = cv2.VideoCapture(f"C:/Users/hagar/OneDrive - mail.tau.ac.il/Desktop/Stage/LPC_2022/Hand decoder/Position/data/test_videos/{fn}.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)


marked_video = cv2.VideoWriter(f'C:/Users/hagar/OneDrive - mail.tau.ac.il/Desktop/Stage/LPC_2022/Hand decoder/Position/results/marked_videos/{fn}_marked.avi',cv2.VideoWriter_fourcc(*'MJPG'),30, size)

fps = cap.get(cv2.CAP_PROP_FPS) 

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    n=0
    while cap.isOpened():
        n = n+1
        ret, frame = cap.read()
        if not ret:
            break
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
       
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        

        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            row = pose_row
            
            # Name colls
            num_coords = len(results.pose_landmarks.landmark)
            col_name = []
            for val in range(1, num_coords+1):
                col_name += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val),'v{}'.format(val)] 


            # Make Detections 
            X = pd.DataFrame([row], columns = col_name)
            
            predicted_position = model.predict(X)[0] # [0] because predict returns 2 arg and we want only the first one
            position_prob = model.predict_proba(X)[0]
            prob_to_display = str(round(position_prob[np.argmax(position_prob)],2))
            print(prob_to_display)
            # Append prediction class and probability 
            row.insert(0, predicted_position)
            row.insert(1, position_prob)

            
            # Write prediction to a CSV
            with open(file_path, mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
                
            # Write prediction on the video
            font = cv2.FONT_HERSHEY_SIMPLEX
            
            # Get status box on the top left corner
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'Predicted Position'
                         , (95,12), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(predicted_position)
                         , (90,40), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

            
            cv2.putText(image, prob_to_display
                        , (10,40), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            
        except Exception as e:
                print(e)
                pass
            


        cv2.imshow('cued_estimated', image)
        marked_video.write(image)


        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


marked_video.release()
cap.release()
cv2.destroyAllWindows()
print("The video was successfully saved")

0.63
0.64
0.65
0.69
0.7
0.46
0.47
0.49
0.59
0.62
0.65
0.63
0.63
0.59
0.61
0.62
0.62
0.65
0.65
0.64
0.65
0.67
0.67
0.65
0.6
0.44
0.45
0.54
0.68
0.7
0.65
0.72
0.73
0.75
0.76
0.76
0.77
0.79
0.79
0.78
0.78
0.8
0.85
0.85
0.83
0.83
0.85
0.86
0.87
0.87
0.87
0.87
0.87
0.87
0.88
0.88
0.85
0.85
0.85
0.85
0.73
0.72
0.71
0.69
0.64
0.63
0.61
0.61
0.6
0.6
0.56
0.56
0.59
0.63
0.62
0.62
0.62
0.64
0.64
The video was successfully saved
